In [1]:
import json
import cv2
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import math
import copy
from scipy.spatial import distance
from pathlib import Path
import scipy

In [2]:
def get_character_plane(data, order=1):
    if order == 1:
        # best-fit linear plane
        A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])    # coefficients
    elif order == 2:
        # best-fit quadratic curve
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
    return [C[0], C[1], -1, C[2]]


def calc_sin_phi(a, b, c):
    return sqrt((a*a + b*b) / (a*a + b*b + c*c))

def calc_cos_phi(a, b, c):
    return c / sqrt(a*a + b*b + c*c)

def calc_u1(a, b, c):
    return b / sqrt(a*a + b*b)


def calc_u2(a, b, c):
    return -a / sqrt(a*a + b*b)

def get_rotation_matrix(plane_coff):
    a, b, c, d = plane_coff
    cos_phi = calc_cos_phi(a, b, c)
    sin_phi = calc_sin_phi(a, b, c)
    u1 = calc_u1(a, b, c)
    u2 = calc_u2(a, b, c)
    rot_matrix = np.array([
        [cos_phi + u1 * u1 * (1 - cos_phi)  , u1 * u2 * (1 - cos_phi)           , u2 * sin_phi  ,  0],
        [u1 * u2 * (1 - cos_phi)            , cos_phi + u2 * u2 * (1 - cos_phi) , -u1 * sin_phi ,  0],
        [-u2 * sin_phi                      , u1 * sin_phi                      ,      cos_phi  , 0],
        [0                                  , 0                                 , 0             ,  1]])
    return rot_matrix

def get_trans_matrix(plane_coff, transform_matrix):
    t_inv = np.linalg.inv(transform_matrix)
    new_plane_coff = np.dot(plane_coff, t_inv)
    t_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, new_plane_coff[3]/new_plane_coff[2]], [0, 0, 0, 1]])
    return t_matrix

def get_transform_matrix(plane_coff):
    rot_matrix = get_rotation_matrix(plane_coff)
    #print(rot_matrix)
    trans_matrix = get_trans_matrix(plane_coff, rot_matrix)
    #print(trans_matrix)
    t_r_matrix = np.matmul(trans_matrix, rot_matrix)
    #print(t_r_matrix)
    return t_r_matrix

def do_transform(np_pc_points, transform_matrix):
    ones = np.ones((np_pc_points.shape[0], 1))
    homo_np_pc_points = np.c_[ np_pc_points, ones]
    homo_pc_3d = transform_matrix@(homo_np_pc_points.T)
    homo_pc_3d = homo_pc_3d.T[:, :3]
    
    mean_vector = np.mean(homo_pc_3d, axis=0)
    if mean_vector[2] < 0:
        # print("inverted")
        homo_pc_3d[:, 2] = - homo_pc_3d[:, 2]
        homo_pc_3d[:, 0] = - homo_pc_3d[:, 0]
        mean_vector = np.mean(homo_pc_3d, axis=0)
    mean_vector[2] = 0
    homo_pc_3d = homo_pc_3d - mean_vector
    return homo_pc_3d

In [106]:
def read_stl_file(pc_path):
    pc_points = o3d.io.read_triangle_mesh(str(pc_path))
    np_pc_points = np.asarray(pc_points.vertices)
    return pc_points, np_pc_points


def transform_characters(plane_point_path, dest_dir_path, woodblock_path):
    # label_dict = read_json(img_anno_file_path)
    # point_index_mapping = read_3d_point_index(pc_anno_file_path)
    # pc_file_path_list = list(Path(pc_dir_path).glob("*.stl"))
    pc_plane_points, np_plane_pc_points = read_stl_file(str(plane_point_path))
    pc_points, np_pc_points = read_stl_file(str(woodblock_path))
    transformed_plane_coff = get_character_plane(np_plane_pc_points)
    transform_matrix = get_transform_matrix(transformed_plane_coff)
    homo_pc_3d = do_transform(np_pc_points, transform_matrix)
    pc_points.vertices = o3d.utility.Vector3dVector(homo_pc_3d)
    pc_points = o3d.geometry.TriangleMesh.compute_triangle_normals(pc_points)
    pc_points.remove_duplicated_vertices()
    o3d.io.write_triangle_mesh(f'{dest_dir_path}/{woodblock_path.stem}_r.stl', pc_points)

### Convert one stl file

In [107]:
pc_point_path = Path("/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/surfaces/08360_kho_khuon_in_1_surface.stl")
dest_dir_path = Path("/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/kho_khuon_in/")
# woodblock_path = Path("/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/29141_kho_khuon_in_1.stl")
#  
woodblock_path = Path("/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/surfaces/08360_kho_khuon_in_1_surface.stl")
transform_characters(pc_point_path, dest_dir_path, woodblock_path)

### Convert directory of stl

In [95]:
woodblock_dir_path = "/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/kho_khuon_in/"
plane_dir_path = "/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/surfaces/"

woodblock_path_list = list(Path(woodblock_dir_path).glob("*.stl"))
plane_path_list = list(Path(plane_dir_path).glob("*.stl"))
woodblock_path_list = sorted(woodblock_path_list, key=lambda x: int(x.stem.split("_")[0]) + int(x.stem.split("_")[-1]));
plane_path_list = sorted(plane_path_list, key=lambda x: int(x.stem.split("_")[0]) + int(x.stem.split("_")[-2]));
woodblock_plane_list = list(zip(woodblock_path_list, plane_path_list))
dest_dir_path = "/mnt/hdd/thuonglc/mocban/taming-transformers/data/woodblock_components/kho_khuon_in_dataset/kho_khuon_in_r/"
for index in range(len(woodblock_path_list)):
    transform_characters(woodblock_plane_list[index][1], dest_dir_path, woodblock_plane_list[index][0])